In [1]:
import os
from groq import Groq
from dotenv import load_dotenv

from agent import Agent

# Load environment variables from .env file
load_dotenv()

True

In [2]:
client = Groq(
    api_key=os.getenv('GROQ_API_KEY'),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)
print(chat_completion.choices[0].message.content)


Fast language models, also known as efficient language models or lite models, have gained significant attention in recent years due to their importance in various applications. Here are some reasons why:

1. **Scalability**: Fast language models are designed to be computationally efficient, allowing them to be deployed on a wide range of devices, from resource-constrained edge devices to powerful servers. This enables them to process large amounts of data and handle a high volume of requests, making them ideal for applications involving real-time processing and decision-making.
2. **Real-time processing**: Fast language models can process and respond to user input in real-time, which is essential for applications like virtual assistants, chatbots, and language translation systems. This capability enables users to interact with these systems seamlessly, without waiting for extended periods of time.
3. **Low-latency conversations**: Fast language models can facilitate low-latency convers

In [3]:

class Agent:
  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = []
    if self.system:
      self.messages.append({"role": "system", "content": self.system})
      
      
  def __call__(self, message=""):
    if message:
      self.messages.append({"role": "user", "content": message})
    result = self.execute()
    self.messages.append({"role": "assistant", "content": result})
    return result

  def execute(self):
    chat_completion = self.client.chat.completions.create(
        model="llama3-70b-8192", messages=self.messages
    )
    return chat_completion.choices[0].message.content


In [4]:
def calculate(operation):
  return eval(operation)

def get_planet_mass(planet) -> float:
    planet_masses = {
        "mercury": 3.3011e23,
        "venus": 4.8675e24,
        "earth": 5.972e24,
        "mars": 6.39e23,
        "jupiter": 1.898e27,
        "saturn": 5.683e26,
        "uranus": 8.681e25,
        "neptune": 1.024e26,
        "pluto": 1.309e22,  # Pluto is considered a dwarf planet
    }
    
    return planet_masses.get(planet.lower(), 0.0)

In [5]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation. The Thought, Action, and PAUSE should output togather
At the end of the loop you output an Answer

Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [6]:
client = Groq(
    api_key=os.getenv('GROQ_API_KEY'),
)

neil_tyson = Agent(client, system_prompt)

In [7]:
result = neil_tyson("What is the mass of Mercury times 5?")
print(result)

Thought: I need to find the mass of Mercury
Action: get_planet_mass: Mercury
PAUSE


In [8]:
result = get_planet_mass("mercury")
print(result)

3.3011e+23


In [9]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: 3.3011e+23'

In [10]:
result = neil_tyson(next_prompt)
print(result)

Thought: I need to multiply this by 5
Action: calculate: 3.3011e+23 * 5
PAUSE


In [11]:
result = calculate("3.285e23 * 5")
print(result)

1.6425e+24


In [12]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: 1.6425e+24'

In [13]:
result = neil_tyson(next_prompt)
result

'Thought: I have the answer\nAnswer: The mass of Mercury times 5 is 1.6525e+24.'

In [14]:
for msg in neil_tyson.messages:
    print(msg)

{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation. The Thought, Action, and PAUSE should output togather\nAt the end of the loop you output an Answer\n\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with t

In [15]:
result = neil_tyson()
print(result)

I apologize for the mistake. The correct answer is:

Answer: The mass of Mercury times 5 is 1.65055e+24.


In [16]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client, system_prompt)
    tools = ['calculate', 'get_planet_mass']
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        print(f"=================================== Iterations: {i} =================================================")
        print(f">> Prompt: {next_prompt}")
        result = agent(next_prompt)
        print(f">> Result: ")
        print(result)
        
        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action:\s*([a-z_]+):\s*(.+)", result, re.IGNORECASE)
            print(f">> Execute the action: {action}")
            chosen_tool = action[0][0]
            arg = action[0][1]
            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"
            print(f">> Get the {next_prompt}")
            continue
        elif "Answer" in result:
            print(f">> Got the point!")
            break

In [17]:
agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of Earth plus the mass of Mercury and all of it times 5?")

=================================== Iterations: 1 =================================================
>> Prompt: What is the mass of Earth plus the mass of Mercury and all of it times 5?
>> Result: 
Thought: I need to find the mass of Earth and Mercury.
Action: get_planet_mass: Earth
PAUSE
>> Execute the action: [('get_planet_mass', 'Earth')]
>> Get the Observation: 5.972e+24
=================================== Iterations: 2 =================================================
>> Prompt: Observation: 5.972e+24
>> Result: 
Thought: I need to find the mass of Mercury.
Action: get_planet_mass: Mercury
PAUSE
>> Execute the action: [('get_planet_mass', 'Mercury')]
>> Get the Observation: 3.3011e+23
=================================== Iterations: 3 =================================================
>> Prompt: Observation: 3.3011e+23
>> Result: 
Thought: I need to add the masses of Earth and Mercury.
Action: calculate: 5.972e+24 + 3.3011e+23
PAUSE
>> Execute the action: [('calculate', '5.972e+24 + 